In [1]:
import ipywidgets as widgets

import geopandas as gpd

from lets_plot.geo_data import *
from lets_plot import *
LetsPlot.setup_html()

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


In [2]:
def display_widgets():
    W, H = 400, 300
    LEVELS = ['country', 'state', 'county', 'city']
    RES = {'country': 3, 'state': 6, 'county': 9}

    changed = True
    w = {
        'country': widgets.Dropdown(
            options=[''] + geocode(level='country').get_geocodes().sort_values('found name')['found name'].to_list(),
            value='',
            description='Country:',
        ),
        'state': widgets.Dropdown(
            options=[],
            description='State:',
        ),
        'county': widgets.Dropdown(
            options=[],
            description='County:',
        ),
        'res': widgets.IntSlider(
            value=1,
            min=1,
            max=15,
            step=1,
            description='Resolution:',
        ),
        'out': widgets.Output(layout=widgets.Layout(height='{0}px'.format(H + 20)))
    }
    q = {'type': None, 'value': None}

    def clear_output(message=None):
        w['out'].outputs = ()
        if message:
            w['out'].append_stdout(message)

    def update_output(res_value=None):
        nonlocal changed
        if not q['type'] or not q['value']:
            clear_output()
        try:
            b_gdf = gpd.GeoDataFrame()
            p_gdf = gpd.GeoDataFrame()
            for i, level in enumerate(LEVELS[:-1]):
                if q['type'] != level or not w[level].value:
                    continue
                scope = None
                for j in range(0, i):
                    scope = geocode(level=LEVELS[j], names=w[LEVELS[j]].value, scope=scope).ignore_all_errors() # w[LEVELS[i-1]].value if i > 0 and w[LEVELS[i-1]].value else None
                if not res_value:
                    res_value = RES[level]
                for j in range(i+1, len(LEVELS)-1):
                    changed = False
                    w[LEVELS[j]].options = []
                if q['value'] == '':
                    b_gdf = geocode(level=level, scope=scope).ignore_all_errors().get_boundaries(res_value)
                else:
                    b_gdf = geocode(level=LEVELS[i+1], scope=w[level].value).ignore_all_errors().get_boundaries(res_value)
                    if LEVELS[i+1] in w.keys():
                        w[LEVELS[i+1]].options = [''] + b_gdf.sort_values('found name')['found name'].to_list()
                    if b_gdf.empty or level == 'county':
                        geocoded_level = geocode(level=level, names=w[level].value, scope=scope)\
                                         .allow_ambiguous().ignore_all_errors()
                        p_gdf = geocode(level='city', scope=geocoded_level).ignore_all_errors().get_centroids()
                        if b_gdf.empty:
                            b_gdf = geocoded_level.get_boundaries(res_value)
                break
            if not b_gdf.empty:
                p = ggplot() + ggsize(W, H)
                if p_gdf.empty:
                    p += geom_map(data=b_gdf, fill='black', color='white', tooltips=layer_tooltips().line('@{found name}'))
                else:
                    p += geom_map(data=b_gdf, fill='black', color='white') + \
                         geom_point(data=p_gdf, shape=1, color='white', tooltips=layer_tooltips().line('@{found name}'))
                w['out'].outputs = ()
                w['out'].append_display_data(p)
            else:
                clear_output()
            changed = False
            w['res'].value = res_value
        except:
            clear_output('Something went wrong')

    def on_change_select(change):
        nonlocal changed
        if change['type'] != 'change' or change['name'] != 'value':
            return
        if not changed:
            changed = True
            return
        q['type'] = change.owner.description[:-1].lower()
        q['value'] = change['new']
        update_output()

    def on_change_slider(change):
        nonlocal changed
        if change['type'] != 'change' or change['name'] != 'value':
            return
        if not changed:
            changed = True
            return
        update_output(change['new'])

    w['country'].observe(on_change_select)
    w['state'].observe(on_change_select)
    w['county'].observe(on_change_select)
    w['res'].observe(on_change_slider)

    return display(*w.values())

display_widgets()

Dropdown(description='Country:', options=('', 'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argenti…

Dropdown(description='State:', options=(), value=None)

Dropdown(description='County:', options=(), value=None)

IntSlider(value=1, description='Resolution:', max=15, min=1)

Output(layout=Layout(height='320px'))